# Сегментация аудио с помощью алгоритма обнаружения активности голоса (VAD)

In [1]:
from pathlib import Path
import subprocess
import pandas as pd
from kaldi.segmentation import NnetSAD, SegmentationProcessor
from kaldi.nnet3 import NnetSimpleComputationOptions
from kaldi.util.table import SequentialMatrixReader

## Определение параметров

In [2]:
MODEL_DIR = Path('../model')
SCP_PATH = 'data/example.scp'
SEGMS_PATH = 'data/segments'
SCP_SEGMS_PATH = 'data/example_segments.scp'
S2U_PATH = 'data/spk2utt'
U2S_PATH = 'data/utt2spk'

In [3]:
decodable_opts = NnetSimpleComputationOptions()
decodable_opts.extra_left_context = 79
decodable_opts.extra_right_context = 21
decodable_opts.extra_left_context_initial = 0
decodable_opts.extra_right_context_final = 0
decodable_opts.frames_per_chunk = 150
decodable_opts.acoustic_scale = 0.3

## Определение модели сегментации

In [4]:
model = NnetSAD.read_model(str(MODEL_DIR / 'final.raw'))
post = NnetSAD.read_average_posteriors(str(MODEL_DIR / 'conf/post_output.vec'))
transform = NnetSAD.make_sad_transform(post)
graph = NnetSAD.make_sad_graph()

In [5]:
sad = NnetSAD(model, transform, graph, decodable_opts=decodable_opts)
seg = SegmentationProcessor([2])

## Выполнение сегментации

In [6]:
feats_rspec = "ark:compute-mfcc-feats --verbose=0 --config=" + str(MODEL_DIR / 'conf/mfcc_hires.conf') + " scp:" + SCP_PATH + " ark:- |"
with SequentialMatrixReader(feats_rspec) as f, open(SEGMS_PATH, 'w') as s:
    for i, (key, feats) in enumerate(f):
        out = sad.segment(feats)
        segs, _ = seg.process(out['alignment'])
        seg.write(key, segs, s)

In [7]:
segments = pd.read_csv(SEGMS_PATH, header=None, sep=' ', names=['Фраза', 'Канал', 'Начало', 'Конец'])
segments

,Фраза,Канал,Начало,Конец
0,example.0-2-0000746-0000832,example.0,7.46,8.32
1,example.0-2-0001277-0001377,example.0,12.77,13.77
2,example.1-2-0000078-0000700,example.1,0.78,7.00
3,example.1-2-0000700-0000886,example.1,7.00,8.86
4,example.1-2-0000940-0001262,example.1,9.40,12.62


## Извлечение сегментов

In [8]:
def make_spk2utt(utt2spk):
    """
    Формирование spk2utt файла
    
    Аргументы:
        utt2spk: путь к файлу сопоставления сегментов и говорящих

    Результат:
        spk2utt: путь к файлу перечисления сегментов для каждого говорящего
    """
    spk2utt = str(Path(utt2spk).parents[0] / 'spk2utt')
    utt2spk_df = pd.read_csv(utt2spk, sep='\t', header=None, names=['utt_id', 'speaker_id'])
    spk2utt_df = utt2spk_df.groupby('speaker_id')['utt_id'].apply(lambda x: ' '.join(x)).reset_index()
    spk2utt_df.to_csv(spk2utt, sep='\t', index=False, header=False)
    return spk2utt

In [9]:
with open(SEGMS_PATH, 'r') as s, \
    open(SCP_SEGMS_PATH, 'w') as ws, \
    open(U2S_PATH, 'w') as u:
    for segment in s:
        segment_info = segment.split(' ')
        segment_id = segment_info[0]
        speaker_id = segment.split(' ')[1].split('.')[-1] or segment_id
        ws.write(segment_id + '\tdata/' + segment_id + '.wav' + '\n')
        u.write(segment_id + '\tКанал ' + speaker_id + '\n')
make_spk2utt(U2S_PATH)
extract_command = "extract-segments scp:" + SCP_PATH + " " + SEGMS_PATH + " scp:" + SCP_SEGMS_PATH
with subprocess.Popen(extract_command, shell=True):
    pass

In [10]:
wav_segments = pd.read_csv(SCP_SEGMS_PATH, header=None, sep='\t', names=['Сегмент', 'Файл'])
wav_segments

,Сегмент,Файл
0,example.0-2-0000746-0000832,data/example.0-2-0000746-0000832.wav
1,example.0-2-0001277-0001377,data/example.0-2-0001277-0001377.wav
2,example.1-2-0000078-0000700,data/example.1-2-0000078-0000700.wav
3,example.1-2-0000700-0000886,data/example.1-2-0000700-0000886.wav
4,example.1-2-0000940-0001262,data/example.1-2-0000940-0001262.wav


In [11]:
utt2spk = pd.read_csv(U2S_PATH, header=None, sep='\t', names=['Сегмент', 'Канал'])
utt2spk

,Сегмент,Канал
0,example.0-2-0000746-0000832,Канал 0
1,example.0-2-0001277-0001377,Канал 0
2,example.1-2-0000078-0000700,Канал 1
3,example.1-2-0000700-0000886,Канал 1
4,example.1-2-0000940-0001262,Канал 1


In [12]:
spk2utt = pd.read_csv(S2U_PATH, header=None, sep='\t', names=['Канал', 'Сегменты'])
spk2utt

,Канал,Сегменты
0,Канал 0,example.0-2-0000746-0000832 example.0-2-000127...
1,Канал 1,example.1-2-0000078-0000700 example.1-2-000070...
